In [1]:
import pandas as pd
import os
import numpy as np
import requests
from datetime import datetime, timedelta
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime, timedelta
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series



In [3]:
if not os.getenv("CI"):  # CI is a common variable set in GitHub Actions
    load_dotenv()
    
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")
entose_api = os.getenv("ENTSOE_API")

# Check if keys are loaded properly
if not hopsworks_api:
    raise ValueError("HOPSWORKS_API_KEY is not set.")
if not entose_api:
    raise ValueError("ENTSOE_API is not set.")

os.environ["ENTSOE_API"] = entose_api
os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")

2025-01-06 14:42:39,456 INFO: Initializing external client
2025-01-06 14:42:39,458 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-06 14:42:40,959 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [4]:
start_date = datetime.now().strftime('%Y-%m-%d')
end_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')

In [5]:
forecast = get_weather_forecast("Stockholm", "2022-11-01", "2025-01-03", 59.3294, 18.0687)

formatted_forecast_df = process_weather_data(forecast)
print(formatted_forecast_df.tail(1))

Coordinates 59.32889938354492°N 18.072357177734375°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                   date  temperature_2m  precipitation  snow_depth  \
167 2025-01-12 23:00:00            -8.9            0.0        0.19   

     pressure_msl  cloud_cover  wind_speed_10m  wind_speed_100m  \
167        1041.0          0.0        6.489992        32.409996   

     wind_direction_10m  wind_direction_100m       city  
167          356.820221           257.816345  Stockholm  


In [6]:
# price_fg = fs.get_feature_group(
#     name='electricity_price_data_3',
#     version=1,
# )
weather_fg = fs.get_feature_group(
    name='weather_data_3',
    version=1,
)
entsoe_fg = fs.get_feature_group(
    name='entsoe_data_3',
    version=1,
)

In [7]:

#formatted_forecast_df['date'] = pd.to_datetime(formatted_forecast_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)

In [7]:
print(formatted_forecast_df.isna().sum())

date                   0
temperature_2m         0
precipitation          0
snow_depth             0
pressure_msl           0
cloud_cover            0
wind_speed_10m         0
wind_speed_100m        0
wind_direction_10m     0
wind_direction_100m    0
city                   0
dtype: int64


In [8]:
formatted_forecast_df = formatted_forecast_df.dropna()
weather_fg.insert(formatted_forecast_df)

2025-01-06 14:42:52,529 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394632


Uploading Dataframe: 100.00% |██████████| Rows 168/168 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_3_1_offline_fg_materialization/executions


(Job('weather_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pressure_msl",
           "min_value": 0.0,
           "max_value": 2000
         },
         "meta": {
           "expectationId": 694524
         }
       },
       "result": {
         "observed_value": 982.6000366210938,
         "element_count": 168,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T01:42:52.000528Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwee

In [9]:
entsoe_df = fetch_historical_data(entose_api, start_date, end_date)

[DEBUG] Successfully fetched load data for finland.
[DEBUG] Successfully fetched load data for norway.
[DEBUG] Successfully fetched load data for denmark.
[DEBUG] Successfully fetched cross-border flows SE3 to finland.
[DEBUG] Successfully fetched cross-border flows finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to norway.
[DEBUG] Successfully fetched cross-border flows norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to denmark.
[DEBUG] Successfully fetched cross-border flows denmark to SE3.


In [10]:
print(entsoe_df['date'].tail(1))

index
2025-01-06 11:00:00+00:00    2025-01-06 12:00:00
Name: date, dtype: object


In [11]:

entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_df.columns = entsoe_df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])
entsoe_df['date'] = pd.to_datetime(entsoe_df['date']).dt.tz_localize('UTC').dt.tz_convert(None)
entsoe_df = entsoe_df.dropna()
print(entsoe_df.dtypes)
print(entsoe_df.tail(5))

load_se3                                                   float64
load_finland                                               float64
load_norway                                                float64
load_denmark                                               float64
prices                                                     float64
total_generation_biomass                                   float64
total_generation_fossil_gas                                float64
total_generation_fossil_hard_coal                          float64
total_generation_fossil_oil                                float64
total_generation_hydro_run_of_river_and_poundage           float64
total_generation_other_renewable                           float64
total_generation_solar                                     float64
total_generation_waste                                     float64
total_generation_wind_offshore                             float64
total_generation_wind_onshore                              flo

In [27]:
#print(entsoe_df['date'].tail(20))

In [16]:
# print(entose_api)
# entsoe_df = fetch_energy_data(entose_api, "2025-01-05", "2025-01-06")

# print(entsoe_df)

In [12]:
entsoe_fg.insert(entsoe_df)

2025-01-06 14:43:08,931 INFO: 	21 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1394633


Uploading Dataframe: 100.00% |██████████| Rows 13/13 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: entsoe_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_3_1_offline_fg_materialization/executions


(Job('entsoe_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "flows_denmark_to_se3",
           "min_value": -0.1,
           "max_value": 10000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 694543
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 13,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T01:43:08.000930Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expec

In [11]:
# price_fg.insert(price)